In [6]:
#%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
import glob
import h5py as h5
from tabulate import tabulate
from datetime import date
import pandas as pd
from os import walk
import peakutils
from peakutils.plot import plot as pplot
from IPython.display import display, clear_output
#sys.path.append('/media/labfiles/lab-codes/lflm-labs-data/src')
from labexp import experiments
plt.style.use('/media/labfiles/lab-codes/plotstyle.mplstyle')
#path=r'/media/labfiles/lab-exps'


In [7]:
expname = '3-wells'
exp = experiments(1,'cry1','ras',expname)
data = exp.data
filesname=exp.filesname
paths=exp.pathname
folders = exp.foldername

|  No. Dir  | Name Dir                                         |  No. files  |
|-----------|--------------------------------------------------|-------------|
|     0     | cry-sys-1/3-wells/exp-2021-12/2021-12-14-ras     |      7      |
|     1     | cry-sys-1/3-wells/exp-2021-12/2021-12-16-ras     |      3      |
|     2     | cry-sys-1/3-wells/exp-2022-01/exp-2022-01-10-ras |      4      |


In [8]:
months = []
days = []
for i in range(len(paths)):
    if os.path.normpath(paths[i]).split(os.path.sep)[7] not in months:
        months.append(os.path.normpath(paths[i]).split(os.path.sep)[7])
    else: pass
    if os.path.normpath(paths[i]).split(os.path.sep)[8] not in days:
        days.append(os.path.normpath(paths[i]).split(os.path.sep)[8])
    else: pass

ld = os.listdir("/media/labfiles/lab-exps/spectro-lab/cry-sys-1/3-wells")
for i in ld:
    if ".opju" in i:
        ld.remove(i)

lend = len(data)
maxdimrows = 0
maxdimcols = 0
dimlist = []
for i in range(len(data)):
    lend2 = len(data[i])
    #print(lend2)
    if lend2 > 0:
        maxdimrows = max([len(j) for j in data[i]])
        maxdimcols = max([data[i][k].shape[1] for k in range(len(data[i]))])
        dimlist.append([maxdimrows,maxdimcols])

allarrs = []
for i in range(len(data)):
    lend2 = len(data[i])
    if lend2 != 0:
        npa = np.empty((dimlist[i][0],dimlist[i][1],lend2), dtype=np.float64)
        for j in range(lend2):
            rows,cols = data[i][j].shape
            npa[0:rows,0:cols,j] = data[i][j]
            
        allarrs.append(npa)

with h5.File(expname+".h5",'a') as file:
    grps = []
    dtsets = []
    for i in range(len(months)):
        group = file.require_group(months[i])
        spltmonths = months[i].split('-',maxsplit=1)
        for j in range(len(days)):
            if spltmonths[1] in days[j]: 
                datas = group.require_dataset(days[j],shape=allarrs[j].shape,data=allarrs[j],dtype=np.float64) 
                dtsets.append(datas)
        grps.append(group)

In [24]:
file = h5.File(expname+'.h5','r')
rs,cs = data[0][0][:].shape
comp = file['exp-2021-12']['2021-12-14-ras'][0:rs,0:cs,0]
print(data[0][0][:]==comp)
file.close()

[[ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]
 [ True  True  True  True]
 